In [ ]:
#Scrapy    
- 비동기 방식으로 데이터 수집 => 데이터 수집 속도가 빠름

In [5]:
import scrapy, requests
from scrapy.http import TextResponse

In [ ]:
# Gmarket 베스트 200 상품 데이터 수집

In [ ]:
# 1. 스크래피 프로젝트 생성

In [1]:
!scrapy startproject gmarket

New Scrapy project 'gmarket', using template directory 'c:\users\user\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\Desktop\김병수\22.08.03-웹 크롤링\code\gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [2]:
!tree gmarket /f

폴더 PATH의 목록입니다.
볼륨 일련 번호가 00000085 7C6C:FC35입니다.
C:\USERS\USER\DESKTOP\김병수\22.08.03-웹 크롤링\CODE\GMARKET
│  scrapy.cfg
│  
└─gmarket
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    └─spiders
            __init__.py
            


In [ ]:
# items.py : 수집할 데이터의 구조 정의
# middlewares.py : 데이터를 수집할 때 headers 정보와 같은 내용을 설정
# pipelines.py : 데이터를 수집한 후에 코드 실행 정의
# settings.py : 크롤링 설정 : 크롤링 시간 텀, robots.txt 규칙
# spiders : 디렉터리 : 크롤링 절차 정의

In [ ]:
# 2. xpath 찾기 : 링크, 상세 페이지의 데이터 

In [ ]:
# 링크 데이터 수집

In [6]:
request = requests.get("http://corners.gmarket.co.kr/Bestsellers")
response = TextResponse(request.url, body=request.text, encoding="utf-8")

In [12]:
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
links[:2]

['http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953128385796138',
 'http://item.gmarket.co.kr/Item?goodscode=2520466046&ver=637953128385796138']

In [ ]:
# 상세 페이지 데이터 수집

In [13]:
link = links[0]
request = requests.get(link)
response = TextResponse(request.url, body=request.text, encoding="utf-8")

In [14]:
title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()')[0].extract()
price = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()')[0].extract()
title, price

('브라운 면도기 세정액 CCR5+1 ', '25,030')

In [ ]:
# 3. item.py : 코드 작성 : model(데이터의 구조 : 수집할 데이터의 컬럼의 정의)

In [17]:
%%writefile gmarket/gmarket/items.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]:
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract()
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        item["link"] = response.url
        yield item

Overwriting gmarket/gmarket/items.py


In [ ]:
# 4. spider.py : 코드 작성 : 크롤링 절차 정의
# 쥬피터 콘솔 창에서 실행해야 됨
# gmarket 디렉터리
# scrapy crawl GMB -o items.csv

In [33]:
cd gmarket

C:\Users\User\Desktop\김병수\22.08.03-웹 크롤링\code\gmarket\gmarket


In [30]:
# 5. 스크래피 프로젝트 실행

#pd.read_csv("gmarket/item.csv").tail(2)